# НОУТБУК ПРЕДНАЗНАЧЕН ДЛЯ ЗАПУСКА БЕЗ ИНТЕРФЕЙСА И ПОКАЗЫВАЕТ РАБОТОСПОСОБНОСТЬ МОДЕЛИ

Для запуска кода нужно наличие следующих таблиц: 

1) Данные о группах "groups.csv"
2) Полная таблица пользователей, в которую были добавлены рандомно сгенерированные имена пользователей 'users_w_fio.csv'
3) Обработанная полная таблица пользователей, которая к использованию (без имен) "main_table.csv"
4) Таблица активностей пользователей "attend.csv"
5) Таблицы - юзеры и группы с координатами "groups_coor.xlsx", "users_itog.xlsx"
6) Таблица пользователей без активности, с рандомно сгенерированными именами "new_users_w_fio.csv"
7) Таблица пользователей без активности, без имен "new_users.xlsx"

In [105]:
#это импорт всего, что может пригодиться и настройка параметров
from sklearn import tree
import pandas as pd
import numpy as np
import geopandas as gpd
import geopy.distance

In [106]:
#подтягиваем таблицы для получения данных 
new_users_w_fio = pd.read_csv("new_users_w_fio.csv",error_bad_lines=False,delimiter=";")
new_users = pd.read_excel("new_users.xlsx")
new_users = new_users.drop(columns="Unnamed: 0")
new_users = new_users.set_index("уникальный номер")
groups_cards = pd.read_csv("groups.csv")
w = pd.read_csv('users_w_fio.csv',error_bad_lines=False,delimiter=";")
users = pd.read_csv("main_table.csv")
users = users.set_index("уникальный номер участника")
attend = pd.read_csv("attend.csv")
gc = pd.read_excel("groups_coor.xlsx")
gc = gc[gc["уникальный номер"].notna()]
uc = pd.read_excel("users_itog.xlsx")
gc["уникальный номер группы"] = gc["уникальный номер"].astype('int64')

# ЕСЛИ ПОЛЬЗОВАТЕЛЬ ЗАРЕГИСТРИРОВАН И РАНЕЕ ХОДИЛ НА ЗАНЯТИЯ

In [108]:
#fio получаем из запроса, если есть фронт
#здесь взято случайное из таблицы
fio = "Апостол Ринат Романович"
#найти уникальный номер, который соответствует фио
user_number=int(w.loc[w['fio'] == fio, 'уникальный номер'])

101363822

In [109]:
t = users.transpose()
norm_t = t.T.reset_index().rename(columns={'уникальный номер участника' : 'id'})
def findKNearestUsers(userCorrCol, k = 228):
    return userCorrCol[userCorrCol.index != userCorrCol.name].nlargest(n = k).index.tolist()
def old_users(user_number):
        kmeans_corr = t.corrwith(t[user_number], method = 'pearson')
        corr = kmeans_corr.to_frame()
        kNeighboors = corr.apply(lambda col: findKNearestUsers(col))
        mask = list(kNeighboors.iloc[:,0])
        dddf = pd.DataFrame(norm_t[norm_t.id.isin(mask)].sum() == 0)
        not_zeros = dddf[dddf[0] == 0].index.tolist()[2:]
        new_ddf = pd.DataFrame(norm_t[norm_t.id.isin(mask)][not_zeros].sum()).reset_index()
        super_ddf = new_ddf[new_ddf['index'].str.contains("направление")].sort_values(by=0, 
                                                                                      ascending=False).rename(columns={0: 'similarity'})
        super_ddf['direction'] = super_ddf['index'].apply(lambda x : x.split('_x_')[1])
        super_attend = attend[['уникальный номер группы', 'направление 3']].drop_duplicates()
        new = super_ddf.merge(super_attend, left_on='direction', right_on='направление 3')
        new = new.drop(columns = ["index","направление 3"])
        dc = pd.merge(new,gc[["coordinate","уникальный номер группы"]],on = "уникальный номер группы")
        dc["coordinate"] = dc["coordinate"].str.replace("[()]","")
        point = uc.loc[uc["уникальный номер"] == user_number, "coordinate"].tolist()
        coor=dc
        coor["distance"] = coor.coordinate.apply(lambda x : geopy.distance.geodesic(point, x).km)
        coor1 = coor.drop_duplicates(subset="direction", keep='first')
        final_set = coor1.sort_values(['similarity', 'distance'],
                  ascending = [False, True])
        final_set = final_set.drop_duplicates(subset="direction", keep='first')
        groups = final_set['уникальный номер группы'][:10].tolist()
        return groups

In [112]:
old_users_groups = old_users(user_number)

<ipython-input-109-1c6e3bd8d184>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  dc["coordinate"] = dc["coordinate"].str.replace("[()]","")


In [113]:
def user_card(old_users_groups):
    card = []
    i = 0
    while i < len(old_users_groups):
        card.append(groups_cards.loc[groups_cards['уникальный номер'] == old_users_groups[i],["направление 3", "адрес площадки"]])
        i+=1
    return card

In [115]:
user_card(old_users_groups)

[                  направление 3  \
150  Оздоровительная гимнастика   

                                                                                             адрес площадки  
150  город Москва, улица Академика Виноградова, дом 12, город Москва, улица Островитянова, дом 22, корпус 2  ,               направление 3  \
11194  Скандинавская ходьба   

                                           адрес площадки  
11194  город Москва, проспект Андропова, дом 46, корпус 1  ,                направление 3  \
9811  Дыхательная гимнастика   

                                            адрес площадки  
9811  г. Москва, проспект Маршала Жукова, дом 76, корпус 2  ,       направление 3                                   адрес площадки
12260         Пение  г. Москва, Карельский бульвар, дом 23, корпус 2,                        направление 3  \
9737  Осваиваем мобильные устройства   

                                      адрес площадки  
9737  г. Москва, Братеевская улица, дом 10, корпус 1  ,     

# ЕСЛИ ПОЛЬЗОВАТЕЛЬ ЗАРЕГИСТРИРОВАН И РАНЕЕ НЕ ХОДИЛ НА ЗАНЯТИЯ

In [116]:
#если данные поступают с фронта, то эта переменная будет принимать их, сейчас это случайное имя из таблицы пользователей без активностей
fio_inactive = "Анаников Петр Николаевич"

In [117]:
user_number=int(new_users_w_fio.loc[new_users_w_fio['fio'] == fio_inactive, 'уникальный номер'])
t_new = new_users.transpose()
t = users.transpose()
norm_t = t.T.reset_index().rename(columns={'уникальный номер участника' : 'id'})

In [118]:
def findKNearestUsers(userCorrCol, k = 228):
    return userCorrCol[userCorrCol.index != userCorrCol.name].nlargest(n = k).index.tolist()

def new_users_activities(user_number):
    kmeans_corr = t_new.corrwith(t_new[user_number], method = 'pearson')
    corr = kmeans_corr.to_frame()
    kNeighboors = corr.apply(lambda col: findKNearestUsers(col))
    mask = list(kNeighboors.iloc[:,0])
    #тут переходим к просто юзерам, другому файлу
    dddf = pd.DataFrame(norm_t[norm_t.id.isin(mask)].sum() == 0)
    not_zeros = dddf[dddf[0] == 0].index.tolist()[2:]
    new_ddf = pd.DataFrame(norm_t[norm_t.id.isin(mask)][not_zeros].sum()).reset_index()
    super_ddf = new_ddf[new_ddf['index'].str.contains("направление")].sort_values(by=0, 
                                                                                  ascending=False).rename(columns={0: 'simulirity'})
    super_ddf['direction'] = super_ddf['index'].apply(lambda x : x.split('_x_')[1])
    super_attend = attend[['уникальный номер группы', 'направление 3']].drop_duplicates()
    new = super_ddf.merge(super_attend, left_on='direction', right_on='направление 3')
    new = new.drop(columns = ["index","направление 3"])
    dc = pd.merge(new,gc[["coordinate","уникальный номер группы"]],on = "уникальный номер группы")
    dc["coordinate"] = dc["coordinate"].str.replace("[()]","")
    point = uc.loc[uc["уникальный номер"] == user_number, "coordinate"].tolist()
    coor=dc
    coor["distance"] = coor.coordinate.apply(lambda x : geopy.distance.geodesic(point, x).km)
    coor1 = coor.drop_duplicates(subset="direction", keep='first')
    final_set = coor1.sort_values(['simulirity', 'distance'],
              ascending = [False, True])
    final_set = final_set.drop_duplicates(subset="direction", keep='first')
    groups = final_set['уникальный номер группы'][:10].tolist()
    return groups

In [119]:
new_users_groups = new_users_activities(user_number)

def new_user_card(new_users_groups):
    card = []
    i = 0
    while i < len(new_users_groups):
        card.append(groups_cards.loc[groups_cards['уникальный номер'] == new_users_groups[i],["направление 3", "адрес площадки"]])
        i+=1

    return card

<ipython-input-118-d97e5764fc5b>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  dc["coordinate"] = dc["coordinate"].str.replace("[()]","")


In [120]:
new_user_card(new_users_groups)

[                              направление 3  \
 3760  ОНЛАЙН Краеведение и онлайн-экскурсии   
 
                                               адрес площадки  
 3760  г. Москва, Зелёный проспект, д. 5/12 стр. 1, офис 123а  ,
                                                 направление 3  \
 1630  ОНЛАЙН Мастер-класс по уходу за кожей в зрелом возрасте   
 
                                                                                                            адрес площадки  
 1630  г. Москва, Студёный проезд, дом 4, корпус 2, город Москва, 1-я Новокузьминская улица, дом 6, корпус 2, квартира 120  ,
                  направление 3  \
 2179  ОНЛАЙН История искусства   
 
                                                           адрес площадки  
 2179  г. Москва, Большая Пионерская улица, д. 15, стр. 1, этаж 1, пом. I  ,
                             направление 3  \
 12589  ОНЛАЙН Различные техники рисования   
 
                                                                     

# ДЛЯ НЕЗАРЕГИСТРИРОВАННЫХ ПОЛЬЗОВАТЕЛЕЙ

In [ ]:
#При работе c интерфейсом, все переменные приходят с ввода пользователя
#Здесь показаны примеры работы без ввода 

In [3]:
district = "Дмитровский" 
#при работе с фронтом район вводится из выадающего списка, не вручную

In [7]:
#рост указывается в метры.сантиметры
height = 1.72
weight = 59
#расчет имт, установка параметров рекомендательной системы по умолчанию и настройка их изменений в зависимости от ввода
bmi=weight/(height*height)
sports = 4
soul = 3
mind = 3
if bmi > 25:
    sports +=1
    mind -+1

1.72
59


In [5]:
groups_cards = groups_cards.drop(columns=["округ площадки","расписание в активных периодах","расписание в плановом периоде"])
groups_cards = groups_cards[groups_cards['район площадки'].notna()]
mask = groups_cards["район площадки"].str.contains("Дмитровский")
groups_cards_district = groups_cards.loc[mask]

In [8]:
#человека спрашивают, сколько часов в день он сидит. A менее 3 B 4-8 C 8+
sit = B

if sit == "A":
    sports -= 2
    soul += 1
    mind += 1
elif sit == "B":
    sports+=1
    mind-=1
else:
    sports+=2
    soul -=1
    mind -=1

B


In [40]:
#человека спрашивают, что он больше хочет - A-Получить новую профессию
#Б-Изучить новый язык
#В-Завести новые знакомства

new = B
if new == "A":
    knowledge = ["Спецпроект / Серебряный университет"]
elif new == "B":
    knowledge =["Английский язык", "Иностранные языки", "ОНЛАЙН Английский язык", "ОНЛАЙН Иностранные языки"]
elif new == "C":
    knowledge = ["Игры", "Спецпроект / Интеллектуальный клуб"]

B


In [13]:
#человека спрашивают, что ему больше нравится
# A- Петь
# Б-Рисовать
# В-Мастерить
# Г - Танцевать
# Д - Смотреть спектакли

leasure = B
if leasure == "A":
    l = ["Пение"]
elif leasure == "B":
    l = ["Рисование"]
elif leasure == "C":
    l = ["Творчество"]
elif leasure == "D":
    l = ["Танцы"]
elif leasure == "E":
    l = ["Спецпроект / Московский театрал"]

B


In [41]:
#человека спрашивают, занимается ли он спортом, варианты ответа Да и нет (y и n)
doing_sports = y
if doing_sports == "y":
    sport_types = ["Спортивные игры", "Фитнес", "Тренажеры", "Коньки", "ГТО", "Лыжи", "Борьба"]
else:
    sport_types = ["ОФП", "Скандинавская ходьба", "Гимнастика" , "ОНЛАЙН ОФП", "ОНЛАЙН Гимнастика"]

n


In [50]:
mask2 = groups_cards_district["направление 1"].str.contains('|'.join(activities))
mask3 = groups_cards_district["направление 3"].str.contains('|'.join(activities))
masked_df = groups_cards_district[~(mask2 & mask3)]
masked_df = masked_df[~masked_df["направление 2"].str.contains("ОНЛАЙН")]
masked_df = masked_df.drop_duplicates(subset=['направление 3'], keep='first')

In [69]:
sport_card = masked_df[masked_df["направление 1"].str.contains("Физическая активность")]
mind_card = masked_df[masked_df["направление 1"].str.contains("Образование")]
soul_card = masked_df[~masked_df["направление 1"].str.contains("Образование|Физическая активность")]

In [73]:
#Рекомендуем вам спортивные активности 
sport_card[:sports]

,уникальный номер,направление 1,направление 2,направление 3,адрес площадки,район площадки,расписание в закрытых периодах
154,801354088,Танцы,Танцы,Бальные танцы,"город Москва, улица Софьи Ковалевской, дом 8А,...","муниципальный округ Дмитровский, муниципальный...","c 09.01.2023 по 31.03.2023, Пн., Ср. 17:00-18:..."
2995,801349046,Танцы,Танцы,Латиноамериканские танцы,"г. Москва, улица Софьи Ковалевской, дом 1, стр...",муниципальный округ Дмитровский,"c 07.04.2022 по 03.10.2022, Вт., Чт. 11:30-12:..."
7748,801353603,Пение,Пение,Хоровое пение,"город Москва, Учинская улица, дом 10, город Мо...","муниципальный округ Дмитровский, муниципальный...","c 09.01.2023 по 31.03.2023, Пн., Ср. 13:45-14:..."


In [121]:
#Рекомендуем вам образовательные активности 
mind_card [:mind]

,уникальный номер,направление 1,направление 2,направление 3,адрес площадки,район площадки,расписание в закрытых периодах
3894,801359805,Образование,Информационные технологии,Осваиваем мобильные устройства,"г. Москва, Карельский бульвар, дом 20, город Москва, Карельский бульвар, дом 20","муниципальный округ Дмитровский, муниципальный округ Дмитровский","c 09.01.2023 по 31.03.2023, Вт., Чт. 16:00-17:00, без перерыва; c 27.09.2022 по 31.12.2022, Вт., Чт. 16:00-17:00, без перерыва; c 13.09.2022 по 26.09.2022, Вт., Чт. 16:30-17:30, без перерыва"
3900,801354082,Образование,Английский язык,Английский язык,"город Москва, улица Софьи Ковалевской, дом 8А",муниципальный округ Дмитровский,"c 09.01.2023 по 31.03.2023, Пн., Ср. 14:30-15:30, без перерыва; c 05.09.2022 по 31.12.2022, Пн., Ср. 14:30-15:30, без перерыва; c 06.04.2022 по 01.09.2022, Ср. 14:45-15:45, без перерыва; c 04.04.2022 по 01.09.2022, Пн. 15:45-16:45, без перерыва"


In [122]:
#Рекомендуем вам активности для души
soul_card[:soul]

,уникальный номер,направление 1,направление 2,направление 3,адрес площадки,район площадки,расписание в закрытых периодах
154,801354088,Танцы,Танцы,Бальные танцы,"город Москва, улица Софьи Ковалевской, дом 8А, г. Москва, улица Софьи Ковалевской, дом 1, г. Москва, Лобненская улица, дом 13А, г. Москва, улица Софьи Ковалевской, дом 18 А","муниципальный округ Дмитровский, муниципальный округ Дмитровский, муниципальный округ Дмитровский, муниципальный округ Дмитровский","c 09.01.2023 по 31.03.2023, Пн., Ср. 17:00-18:00, без перерыва; c 05.09.2022 по 31.12.2022, Пн., Ср. 17:00-18:00, без перерыва; c 16.03.2022 по 01.09.2022, Пн., Ср. 16:00-17:00, без перерыва; c 10.01.2022 по 15.03.2022, Пн., Ср. 16:00-17:00, без перерыва; c 01.09.2021 по 31.12.2021, Пн., Ср. 16:00-17:00, без перерыва; c 09.08.2021 по 31.08.2021, Пн., Ср. 11:00-12:00, без перерыва; c 02.08.2021 по 08.08.2021, Пн., Ср. 11:00-12:00, без перерыва; c 02.09.2020 по 31.12.2020, Пн., Ср. 16:00-17:00, без перерыва; c 10.09.2019 по 31.12.2019, Вт., Чт. 18:00-19:00, без перерыва; c 09.01.2020 по 01.09.2020, Вт., Чт. 18:00-19:00, без перерыва"
2995,801349046,Танцы,Танцы,Латиноамериканские танцы,"г. Москва, улица Софьи Ковалевской, дом 1, строение 1",муниципальный округ Дмитровский,"c 07.04.2022 по 03.10.2022, Вт., Чт. 11:30-12:30, без перерыва"
7748,801353603,Пение,Пение,Хоровое пение,"город Москва, Учинская улица, дом 10, город Москва, Учинская улица, дом 10","муниципальный округ Дмитровский, муниципальный округ Дмитровский","c 09.01.2023 по 31.03.2023, Пн., Ср. 13:45-14:45, без перерыва; c 27.09.2022 по 31.12.2022, Пн., Ср. 13:45-14:45, без перерыва; c 05.09.2022 по 26.09.2022, Пн., Ср. 13:45-14:45, без перерыва; c 01.06.2022 по 01.09.2022, Ср. 12:30-14:30, без перерыва; c 04.04.2022 по 31.05.2022, Пн., Ср. 12:30-13:30, без перерыва"
